# MILP Formulation from Nathan Kallus' Paper (Problem 4)

In [1]:
import gurobipy as gp
from gurobipy import GRB
import sys
import math
import numpy as np
import pandas as pd
import random

## Model specifications
#### Since we chose to modify the formulation to a certain extent, these variables simply allow us to revert back to the original model
- delta_include: If true, then constraint 4c from original formulation holds. If false, only the added constraint that gamma[p] need to add to 1 holds
- different_Cp = If true, then we have different sets of branching choices for every non-leaf node (like original formulation). If false, then we have a static set C for all nodes

In [2]:
# -- This class is an alternative to solving the right/left ancestor problem --
"""
INPUT: d = depth of tree (which includes root node), so d = 2 would make a tree with {1, 2, 3}
RELEVANT FUNCTIONS:
- get_right_left: For all leaf nodes, returns its right and left ancestors in a dictionary 
                  of {(p, q): 1 or -1 if q is right or left ancestor respectively}
"""
class Tree:
    def __init__(self, d):
        self.depth = d
        self.nodes = list(range(1, 2**(d-1)))
        self.leaves = list(range(2**(d-1), 2**d))
        self.ancestor_rl = {}
    
    def get_left_children(self, n):
        if n in self.nodes:
            return int(2*n)
        else:
            raise Exception ('Invalid node n')
    
    def get_right_children(self, n):
        if n in self.nodes:
            return int(2*n+1)
        else:
            raise Exception ('Invalid node n')
    
    def get_parent(self, n):
        if (n in self.nodes) | (n in self.leaves):
            return int(math.floor(n/2))
        else:
            raise Exception ('Invalide node n')
    
    def get_ancestors(self, direction, n):
        current = n
        ancestors = []
        while current != 1:
            current_buffer = self.get_parent(current)
            if direction == 'r':
                if self.get_right_children(current_buffer) == current:
                    ancestors.append(current_buffer)
            else:
                if self.get_left_children(current_buffer) == current:
                    ancestors.append(current_buffer)
            current = current_buffer
        return ancestors
    
    def get_right_left(self):
        for i in self.leaves:
            right = self.get_ancestors('r', i)
            for j in right:
                self.ancestor_rl[(i, j)] = 1
            left = self.get_ancestors('l', i)
            for j in left:
                self.ancestor_rl[(i, j)] = -1
        return self.ancestor_rl

# Evaluating Synthetic Data

In [3]:
def open_file(file_path):
    df = pd.read_csv(file_path)
    #train_X = df.iloc[:, :25]
    train_X = df[['Age1.2', 'Age3.4', 'Age5.6', 'Age7', 'Age8.9', 'Height1', 'Height2', 'Height3', 'Height4', 'Height5',
                'Weight1', 'Weight2', 'Weight3', 'Weight4', 'Weight5', 'Asian', 'Black.or.African.American', 'Unknown.Race',
                'X.1..1', 'X.1..3', 'X.2..2', 'X.2..3', 'X.3..3', 'Unknown.Cyp2C9', 'VKORC1.A.G', 'VKORC1.A.A', 'VKORC1.Missing']]
    real = df[['y0', 'y1', 'y2']]
    train_t = df['t']
    train_y = df['y']
    return train_X, train_t, train_y, real

In [5]:
def open_file_v2(file_path):
    df = pd.read_csv(file_path)
    train_X = df.iloc[:, :3]
    real = df.iloc[:, 3:5]
    train_t = df.iloc[:, 5]
    train_y = df.iloc[:, 7]
    return train_X, train_t, train_y, real

In [24]:
def datapoint_tree(node, i, test_X, test_real, test_t):
    if node in L: #if datapoint has reached leaf node, calculate error
        index = treatments[node]
        ideal_outcome = max(test_real.iloc[i, :])
        difference = ideal_outcome - test_real.iloc[i, int(index)]
        #print(test_real.iloc[i, index])
        if difference == 0:
            count_optimal = 1
        else:
            count_optimal = 0
        
        if index == test_t[i]:
            same_treatment = 1
        else:
            same_treatment = 0
        return difference, count_optimal, same_treatment
    if test_X.iloc[i, branching[node]] <= 0: # go left (node 2)
        return datapoint_tree(tree.get_left_children(node), i, test_X, test_real, test_t)
    else:
        return datapoint_tree(tree.get_right_children(node), i, test_X, test_real, test_t)

In [5]:
def get_metrics(test_X, test_real, test_t):
    difference = 0
    count_optimal = 0
    count_same = 0
    for i in range(len(test_X)):
        diff, optimal, treat = datapoint_tree(1, i, test_X, test_real, test_t)
        difference += diff
        count_optimal += optimal
        count_same += treat
    return difference, float(count_optimal)/len(test_X), float(count_same)/len(test_X)

## Declaring variables determined a-priori

### a) Specfiying Input to Model
- m treatments indexed by t {1,..., m}
- n datapoints indexed by i {(X1, T1, Y1), ..., (Xn, Tn, Yn)} 
- d: depth of decision tree
- n_min: minimum number of datapoints of each treatment in node p
- num_features
- num_cuts

In [25]:
def run_tree(depth, bert):
    delta_include = False
    different_Cp = False
    bertsimas = bert

    m = {0, 1, 2}
    n = len(train_X)
    d = int(depth)
    n_min = 0
    num_features = 2
    num_cuts = 2

    """# ---- CONSTRUCTING COMPLETE BINARY TREE ----
    # - P = number of nodes in the tree
    # - L_c = set of non-leaf nodes
    # - L = set of leaf ndoes"""

    P = 2**d
    L_c = set(range(1, 2**(d-1)))
    L = set(range(2**(d-1), P))

    # - ancestors: dictionary {leaf nodes: {set of ancestors}}
    ancestors = {}
    for p in L:
        ancestors[p] = [math.floor(p/(2**j)) for j in range(1, d)]

        # Alternative way of retrieving right/left ancestors
    tree = Tree(d)
    right_left = tree.get_right_left()


    """
    - C[p]: finite set of cuts on node p--determined apriori {(l, theta)}
        Representation: dictionary {non-leaf node: list of (l, theta)}
        Require a list instead of set so it's ordered (indexed easily)


    From Kallus' paper, he wants us to:
    1. For each l in [d], sort data along x_l
    2a. For each non-leaf node, pick #features from [d] randomly
    2b. Set J = {1, (n-1/#cuts), ..., n-1} in decreasing order of cuts
    2c. Set Cp = {(l, midpoint between the two buckets in J) for all dimensions chosen and for all j in J}

    Make version of ALG3 to take all features"""


    if different_Cp:
        pass
    else:
        # -- BINARY COVARIATES --> Create a finite set of cuts for C for all features --
        C = []
        for i in range(len(train_X.columns)):
            C.append((i, 0))


    """ --- OTHER DATA --- 
    BIG M Constraints:
    - Ybar
    - Ymax
    - M

    BINARY ENCODING FOR CUTS
    - k_p: dictionary {non-leaf node p: k_p value}
    - Z_p: dictionary {non-leaf node p: k_p x |C_p| 2d matrix}
    """

    # ---- Big M Constraints ----
    # Ybar is merged into data, but it could not be. ybar is a numpy array
    minimum = min(train_y)

    ybar = train_y - minimum
    #data['ybar'] = ybar

    # Ymax
    ymax = max(ybar)

    # M
    # Find all sums for treatments 1, ..., m
    #treatment_counts = treatment.value_counts().to_list()
    unique, counts = np.unique(train_t, return_counts=True)
    #frequencies = numpy.asarray((unique, counts)).T
    M = np.array(counts)
    M -= len(L) * n_min
    M = max(M)
    
    model = gp.Model("Kallus")
    #model.params.TimeLimit = 3600

    # -- VARIABLE DECLARATION --

    # -- Variables to determine: gamma and lambda --
    # 1. gamma_p = choice of cut at node p ([0, 1]^C_p) (only applies to non-leaf node)
    #       - represent with a matrix gamma (|L_c| x |C_p|)

    gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 
    # This assumes gamma is binary
    #gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 

    # 2. lambda_pt = choice of treatment t at node p (only applies to leaf nodes L)
    #       - represent with a matrix lamb (|L| x m)
    lamb = model.addVars(L, m, vtype=GRB.BINARY, name='lamb')


    # -- Other Variables in Formulation --
    # 1. w_ip = membership of datapoint i in node p (only applies to leaf nodes L)
    #       - represent with a matrix w (n x |L|)

    w = model.addVars(n, L, lb=0, ub=1, name='w')
    # This assumes w is binary, when in reality it is continuous from 0-1
    #w = model.addVars(n, L, vtype=GRB.BINARY, name='w') # Original paper has this be a continuous variable

    # 2. mu_p = mean outcome of prescribed treatment in node p
    #       - represent with a matrix mu (|L|)
    mu = model.addVars(L, lb=0, name='mu') # define in constraint

    # 3. nu_ip = "effect" of treatment in node p by multiplying mu and w
    #       - represent with a matrix nu (n x |L|)
    nu = model.addVars(n, L, lb=0, name='nu')

    # 4. delta_p = forces only 1 choice of cut at node p
    #       - represent with a dictionary {non-leaf node p: 1d matrix of size k_p}
    #delta = model.addVars(L, k, vtype=GRB.BINARY, name='delta')

    # 5. Chi_i(gamma) = 1 if choice of cut induces datapoint i to go left on the cut gamma, 0 otherwise
    chi = model.addVars(L_c, n, vtype=GRB.BINARY, name='chi')

    if bertsimas:
        # 6. f_i
        f = model.addVars(n, lb=0, name='f')

        # 7. Beta_lt
        beta = model.addVars(L, m, lb=0, name='beta')

    theta = 0.5
    
        # --- OBJECTIVE FUNCTION ---
    if bertsimas:
        model.setObjective(theta * gp.quicksum(nu[i, p] for i in range(n) for p in L) 
                       - (1-theta) * gp.quicksum((train_y[i] - f[i]) * (train_y[i] - f[i]) for i in range(n)), GRB.MAXIMIZE)
    else:
        model.setObjective(gp.quicksum(nu[i, p] for i in range(n) for p in L), GRB.MAXIMIZE)


    # --- CONSTRAINTS ---
    # Constraint 4c (4b is done by definition of variables)
    if delta_include:
        for p in L_c:
            # need to do matrix multiplication somehow, but this might work?
            for j in range(k):
                model.addConstr(delta[p, j] == gp.quicksum(gamma[p, i] * z[j, i] for i in range(len(C))))

    # Additional constraint that gamma[p] adds up to 1
    # CHECKED
    for p in L_c:
        model.addConstr(gp.quicksum(gamma[p, i] for i in range(len(C))) == 1)

    # Add constraint Chi
    for i in range(n):
        for p in L_c:
            model.addConstr(chi[p, i] == gp.quicksum(gamma[p, j] for j in range(len(C)) if C[j][1] >= train_X.iloc[i, C[j][0]]))


    # Constraint 4d&e (Membership restriction from its ancestors) CHECKED
    for p in L:
        A_p = ancestors[p] #index ancestors of p
        for q in A_p:
            R_pq = right_left[(p, q)]
            for i in range(n):
                model.addConstr(w[i, p] <= (1+R_pq)/2 - R_pq * chi[q, i])


    #4e CHECKED
    for p in L:
        A_p = ancestors[p] #index ancestors of p
        for i in range(n):
            model.addConstr(w[i, p] >= 1 + gp.quicksum(-chi[q, i] for q in A_p if right_left[(p, q)] == 1)
                        + gp.quicksum(-1+chi[q, i] for q in A_p if right_left[(p, q)] == -1))


    # Constraint 4f
    # CHECKED
    for t in m:
        for p in L:
            model.addConstr(gp.quicksum(w[i, p] for i in range(n) if train_t[i] == t) >= n_min) #assuming the input comes in vector (Xi, Ti, Yi)
            # only add the datapoints that have been given treatment t

    # Constraints 4g&h (Linearization of nu)
    # CHECKED
    for p in L:
        for i in range(n):
            model.addConstr(nu[i, p] <= ymax * w[i, p])
            model.addConstr(nu[i, p] <= mu[p])
            model.addConstr(nu[i, p] >= mu[p] - ymax * (1-w[i, p]))

    # Constraint 4i (Choice of treatment applied to p)
    # CHECKED
    for p in L:
        model.addConstr(gp.quicksum(lamb[p, t] for t in m) == 1)

    # Constraint 4j&k (Consistency between lambda and mu)
    # CHECKED. There are some inconsistencies where some w don't appear, but this is because ybar is 0 (i.e. the minimum)
    for p in L:
        for t in m:
            model.addConstr(gp.quicksum(nu[i, p] - w[i, p] * ybar[i] for i in range(n) if train_t[i] == t) <= M*(1-lamb[p, t]))
            model.addConstr(gp.quicksum(nu[i, p] - w[i, p] * ybar[i] for i in range(n) if train_t[i] == t) >= M*(lamb[p, t]-1))
    #model.addConstr(lamb[2, 0] == 1)
    if bertsimas:
        for i in range(n):
            for p in L:
                for t in m:
                    if train_t[i] == t:
                        model.addConstr(f[i] - beta[p, t] <= M * (1-w[i, p]))
                        model.addConstr(f[i] - beta[p, t] >= M * (w[i, p]-1))
    
    #model.params.TimeLimit = 3600
    timeLimit = 3600
    oldSolutionLimit = model.Params.SolutionLimit
    model.Params.SolutionLimit = 1
    model.optimize()
    model.Params.TimeLimit = timeLimit - model.getAttr(GRB.Attr.Runtime)
    model.Params.SolutionLimit = oldSolutionLimit - model.Params.SolutionLimit
    model.optimize()
    
    
    #model.optimize()
    g = model.getAttr("X", gamma).items()
    l = model.getAttr("X", lamb).items()
    
    branching = {i[0][0]: i[0][1] for i in g if i[1] == 1.0}
    treatments = {i[0][0]: i[0][1] for i in l if i[1] == 1.0}
    
    return branching, treatments


In [7]:
"""kallus = pd.DataFrame(columns=['Dataset','Depth','P(Correct Treatment)','Test Error','Test % Optimal',
                               'Test % Same Treatment', 'Train Error', 'Train % Optimal', 
                               'Train % Same Treatment', 'Tree'])
bertsimas = pd.DataFrame(columns=['Dataset','Depth','P(Correct Treatment)','Test Error','Test % Optimal',
                               'Test % Same Treatment', 'Train Error', 'Train % Optimal', 
                                  'Train % Same Treatment', 'Tree'])"""


#bertsimas = pd.read_csv('bertsimas_athey_500.csv')

FileNotFoundError: [Errno 2] File b'bertsimas_athey_500.csv' does not exist: b'bertsimas_athey_500.csv'

In [26]:
kallus = pd.DataFrame(columns=['Dataset','Depth','P(Correct Treatment)','Test Error','Test % Optimal',
                               'Test % Same Treatment', 'Train Error', 'Train % Optimal', 
                               'Train % Same Treatment', 'Tree'])


prob = 0.5
bert = False

depths = [1, 2, 3]
datasets = [1, 2, 3, 4, 5]
#probs = [0.1, 0.5, 0.9]

for dataset in datasets:
    for d in depths:
        train_filepath = 'data/Warfarin_2000/data_train_enc_' + str(dataset) + '.csv'
        test_filepath = 'data/Warfarin_2000/data_test_enc_' + str(dataset) + '.csv'
        train_X, train_t, train_y, train_real = open_file(train_filepath)
        branching, treatments = run_tree(d, bert)

        tree = Tree(d)
        L = set(range(2**(d-1), 2**d))
        test_X, test_t, test_y, test_real = open_file(test_filepath)
        error, pct, acc = get_metrics(test_X, test_real, test_t)
        error1, pct1, acc1 = get_metrics(train_X, train_real, train_t)

        tree_stats = 'branching = ' + str(branching) + ', treatments = ' + str(treatments)
        row = [dataset, d, prob, error, pct*100, acc*100, error1, pct1*100, acc1*100, tree_stats]
        print(row)
        if bert:
            bertsimas.loc[len(bertsimas)] = row
        else:
            kallus.loc[len(kallus)] = row


Changed value of parameter SolutionLimit to 1
   Prev: 2000000000  Min: 1  Max: 2000000000  Default: 2000000000
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8010 rows, 4004 columns and 23359 nonzeros
Model fingerprint: 0x23a4e204
Variable types: 4001 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 8010 rows and 4004 columns
Presolve time: 0.06s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 1067.28 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.067278287462e+03, best bound 1.067278287462e+03, gap 0.0000%
Changed value of parameter TimeLimit to 3599.933007955551
   Prev: inf  Min: 0.0  Max: in

H   63    53                    1211.9406618 3590.85088   196%  1847  946s
    73    60 2327.22353    4 2971 1211.94066 3215.50000   165%  1709  957s
    83    65 2397.00000    4 1985 1211.94066 3215.50000   165%  1588  967s
    92    73 2000.00000    5 1580 1211.94066 3215.50000   165%  1501  971s
   100    81 2000.00000    6 1580 1211.94066 3215.50000   165%  1430  976s
H  104    81                    1212.3833184 3215.50000   165%  1392  976s
*  106    81              23    1212.8626925 3215.50000   165%  1406  976s
   133   111 1229.95785   13    2 1212.86269 3215.50000   165%  1277  980s
H  143   111                    1214.0293591 3215.50000   165%  1220  980s
H  144   111                    1277.4265828 3215.50000   152%  1212  980s
   154   115 1562.69661   13  701 1277.42658 2865.88256   124%  1185  985s
H  184   120                    1277.5044456 2865.88256   124%  1113  988s
   188   123 1788.08424    9 1889 1277.50445 2865.88256   124%  1107  992s
   207   132 1649.77796  

  5822  1027 1441.44182   26  705 1290.40341 1967.92977  52.5%   597 1909s
  5951  1016     cutoff   33      1290.40341 1967.92977  52.5%   593 1920s
  6081   988 1330.48873   20  846 1290.40341 1967.92977  52.5%   589 1932s
  6169   992 1913.12636   23  381 1290.40341 1967.92977  52.5%   587 1944s
  6307   962 1357.87907   28  691 1290.40341 1967.92977  52.5%   585 1953s
  6426   945 1611.04963   24 1091 1290.40341 1967.92977  52.5%   582 1972s
  6481   941 1820.59746   19  385 1290.40341 1967.92977  52.5%   581 1983s
  6590   916     cutoff   22      1290.40341 1967.92977  52.5%   579 1994s
  6692   914 1640.75355   27 1214 1290.40341 1967.92977  52.5%   578 2007s
  6852   875     cutoff   25      1290.40341 1967.92977  52.5%   574 2020s
  6971   857     cutoff   22      1290.40341 1967.92977  52.5%   574 2033s
  7104   880     cutoff   26      1290.40341 1967.92977  52.5%   572 2048s
  7263   892 1477.69666   22  498 1290.40341 1967.92977  52.5%   571 2068s
  7407   893 1816.61362  

Changed value of parameter SolutionLimit to 1999999999
   Prev: 1  Min: 1  Max: 2000000000  Default: 2000000000
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 22021 rows, 10035 columns and 93393 nonzeros
Model fingerprint: 0xda12996c
Variable types: 8002 continuous, 2033 integer (2033 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolved: 12445 rows, 5302 columns, 60030 nonzeros

Continuing optimization...

H    0     2                     654.9196134 2000.00000   205%     -    1s
     0     2 2000.00000    0    5  654.91961 2000.00000   205%     -    2s
     1     4 2000.00000    1 1203  654.91961 2000.00000   205%  44.0   79s
     3     8 2000.00000    2 1206  654.91961 2000.00000   205%   358   81s
H    5     8                    1125.3815951 2000.00000  77

  1447   493     cutoff   11      1293.33807 2192.00000  69.5%   690 1112s
  1475   499     cutoff   12      1293.33807 2135.59524  65.1%   700 1120s
  1539   499 1399.95883   14  614 1293.33807 2120.93049  64.0%   698 1133s
  1594   541 1522.03408   15 1622 1293.33807 2101.18009  62.5%   702 1145s
  1686   546 1808.64984   12 2158 1293.33807 2100.50000  62.4%   701 1158s
  1751   549     cutoff   15      1293.33807 2090.44021  61.6%   703 1171s
  1826   560 1753.83396   14 1672 1293.33807 2049.23256  58.4%   699 1183s
  1881   562 1442.41772   12 1584 1293.33807 2032.28571  57.1%   700 1195s
  1933   575 1696.95041   13 1466 1293.33807 2032.28571  57.1%   704 1224s
  1964   582     cutoff   13      1293.33807 2032.28571  57.1%   702 1237s
  2037   620     cutoff   15      1293.33807 2032.00000  57.1%   701 1255s
* 2110   620              26    1293.7934183 2003.67181  54.9%   699 1255s
  2171   683 1336.94534   20 1646 1293.79342 2000.00000  54.6%   695 1282s
  2405   749 1708.88369  

...


H    0     0                     254.2113323 4000.00000  1473%     -    0s

Explored 0 nodes (1729 simplex iterations) in 0.95 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 254.211 

Solution limit reached
Best objective 2.542113323124e+02, best bound 4.000000000000e+03, gap 1473.4940%
Changed value of parameter TimeLimit to 3599.0469069480896
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter SolutionLimit to 1999999999
   Prev: 1  Min: 1  Max: 2000000000  Default: 2000000000
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 22021 rows, 10035 columns and 93126 nonzeros
Model fingerprint: 0x9bd41793
Variable types: 8002 continuous, 2033 integer (2033 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolved: 12541 r

   910   287 1863.19834    7 1369 1284.00482 2141.08688  66.8%   835 1554s
   957   292     cutoff   10      1284.00482 2141.08688  66.8%   829 1563s
  1000   299     cutoff   13      1284.00482 2105.94772  64.0%   825 1572s
  1071   303 1557.05349    9 1706 1284.00482 2096.59651  63.3%   801 1583s
  1110   320 1895.58366    9 1107 1284.00482 2084.00000  62.3%   804 1591s
  1163   327 1355.87118   16  545 1284.00482 2016.00028  57.0%   795 1601s
  1228   343 1461.80077   14  984 1284.00482 2000.00000  55.8%   792 1614s
  1300   408 1343.37165   17  227 1284.00482 2000.00000  55.8%   786 1643s
  1559   473 1524.14451   16  260 1284.00482 2000.00000  55.8%   731 1664s
  1741   512 1407.61477   13 1548 1284.00482 2000.00000  55.8%   720 1685s
  1904   591 1625.25732   10 1519 1284.00482 2000.00000  55.8%   709 1708s
  2100   592 1474.73651   11 1287 1284.00482 2000.00000  55.8%   704 1756s
  2102   593 2000.00000    9 2493 1284.00482 2000.00000  55.8%   704 1760s
  2104   595 1566.13208  

 18497  2991     cutoff   23      1284.00482 1907.40909  48.6%   516 3316s
 18836  3047 1449.28017   29  512 1284.00482 1900.00000  48.0%   516 3343s
 19220  3065 1320.74408   30  543 1284.00482 1895.40318  47.6%   515 3372s
 19576  3103     cutoff   23      1284.00482 1893.04278  47.4%   514 3410s
 19929  3134 1382.15031   23  618 1284.00482 1885.39723  46.8%   513 3446s
 20324  3168 1363.69585   28  618 1284.00482 1876.68531  46.2%   512 3478s

Explored 20783 nodes (10586502 simplex iterations) in 3598.73 seconds
Thread count was 8 (of 8 available processors)

Solution count 10: 1284 1282.04 1252.53 ... 1126.81

Time limit reached
Best objective 1.284004822846e+03, best bound 1.870588235294e+03, gap 45.6839%
[4, 3, 0.5, 874, 63.3696563285834, 34.03185247275775, 697, 65.14999999999999, 32.9, 'branching = {1: 1, 2: 25, 3: 12}, treatments = {4: 1, 5: 0, 6: 1, 7: 1}']
Changed value of parameter SolutionLimit to 1
   Prev: 2000000000  Min: 1  Max: 2000000000  Default: 2000000000
Gurobi Op

     1     4 2674.47918    1 2570 1151.33037 3965.18750   244% 38591  700s
     3     8 2614.75183    2 2279 1151.33037 3589.91667   212% 15375  711s
H    7    12                    1252.5710051 3391.50000   171%  8152  717s
    15    20 2000.00000    4  433 1252.57101 3252.00000   160%  4741  724s
H   17    20                    1300.9935242 3252.00000   150%  4236  724s
    19    24 2000.00000    5    9 1300.99352 3252.00000   150%  4207  726s
    43    43     cutoff   11      1300.99352 3252.00000   150%  2132  730s
H   66    50                    1306.8645260 3252.00000   149%  1706  732s
    72    60     cutoff   14      1306.86453 3251.80000   149%  1652  736s
   101    68 1825.65116   10    4 1306.86453 3251.80000   149%  1347  740s
   137    78     cutoff   16      1306.86453 3251.80000   149%  1150  746s
   177    84     cutoff   14      1306.86453 3251.80000   149%  1015  751s
   200    96     cutoff   13      1306.86453 3094.00000   137%   987  758s
   215    97 2657.00000  

  6061  1007 1506.95522   20  575 1324.70475 2000.00000  51.0%   551 1499s
  6329  1061 1366.56000   24    2 1324.70475 2000.00000  51.0%   543 1512s
  6582  1093     cutoff   26      1324.70475 2000.00000  51.0%   537 1524s
  6734  1134     cutoff   27      1324.70475 2000.00000  51.0%   536 1538s
  6968  1158 1946.06275   20 1456 1324.70475 2000.00000  51.0%   532 1562s
  7066  1203     cutoff   27      1324.70475 2000.00000  51.0%   531 1577s
  7309  1215 1637.00000   24  748 1324.70475 2000.00000  51.0%   530 1591s
  7387  1258     cutoff   26      1324.70475 2000.00000  51.0%   530 1605s
  7626  1314 1341.35527   19  893 1324.70475 2000.00000  51.0%   529 1624s
  7881  1418 1749.20176   30  674 1324.70475 2000.00000  51.0%   526 1643s
  8166  1440 1661.00000   24  127 1324.70475 2000.00000  51.0%   524 1663s
  8496  1512 1513.97143   17  779 1324.70475 2000.00000  51.0%   520 1681s
  8791  1571 2000.00000   23  677 1324.70475 2000.00000  51.0%   519 1702s
  9088  1667 1548.70158  

In [27]:
kallus.to_csv('Results_Warfarin/kallus.csv')

In [11]:
dataset = 4
d = 2
bert = False
train_filepath = 'data/IST_2000/data_train_enc_' + str(dataset) + '.csv'
test_filepath = 'data/IST_2000/data_test_enc_' + str(dataset) + '.csv'
train_X, train_t, train_y, train_real = open_file(train_filepath)
branching, treatments = run_tree(d, bert)

tree = Tree(d)
L = set(range(2**(d-1), 2**d))
test_X, test_t, test_y, test_real = open_file(test_filepath)
error, pct, acc = get_metrics(test_X, test_real, test_t)
error1, pct1, acc1 = get_metrics(train_X, train_real, train_t)

tree_stats = 'branching = ' + str(branching) + ', treatments = ' + str(treatments)
row = [dataset, d, error, pct*100, acc*100, error1, pct1*100, acc1*100, tree_stats]
print(row)

Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 22039 rows, 10039 columns and 99696 nonzeros
Model fingerprint: 0xb0b31bf7
Variable types: 8002 continuous, 2037 integer (2037 binary)
Coefficient statistics:
  Matrix range     [3e-02, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-01, 6e+02]
Presolve removed 8034 rows and 4022 columns
Presolve time: 0.58s
Presolved: 14005 rows, 6017 columns, 79167 nonzeros
Variable types: 4002 continuous, 2015 integer (2015 binary)

Root relaxation: objective 1.897781e+03, 7002 iterations, 0.98 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1897.78083    0  568          - 1897.78083  

In [76]:
tree = Tree(d)
L = set(range(2**(d-1), 2**d))
test_X, test_t, test_y, test_real = open_file(test_filepath)
error, pct, acc = get_metrics(test_X, test_real, test_t)
error1, pct1, acc1 = get_metrics(train_X, train_real, train_t)
print(error, pct, acc)
print(error1, pct1, acc1)

267.9863312540136 0.4446 0.5066
21.252969854711775 0.354 0.5


In [77]:
tree_stats = 'branching = ' + str(branching) + ', treatments = ' + str(treatments)
row = [dataset, d, prob, error, pct*100, acc*100, error1, pct1*100, acc1*100, tree_stats]
if bert:
    bertsimas.loc[len(bertsimas)] = row
else:
    kallus.loc[len(kallus)] = row

[5, 3, 0.9, 267.9863312540136, 44.46, 50.660000000000004, 21.252969854711775, 35.4, 50.0, 'branching = {1: 10, 2: 16, 3: 0}, treatments = {4: 1, 5: 1, 6: 1, 7: 1}']


In [78]:
print(bertsimas)
bertsimas.to_csv('bertsimas_athey_500.csv', index=False)

    Dataset  Depth  P(Correct Treatment)  Test Error  Test % Optimal  \
0         1      2                   0.5   53.227602           76.83   
1         1      3                   0.5  123.833930           66.00   
2         1      3                   0.5  123.833930           66.00   
3         2      2                   0.5   24.915258           84.31   
4         2      3                   0.5   24.915258           84.31   
5         3      2                   0.5  166.280038           58.61   
6         3      3                   0.5   57.544135           77.50   
7         4      2                   0.5  432.810829           24.05   
8         4      3                   0.5  231.607156           44.69   
9         5      2                   0.5  189.645268           55.55   
10        5      3                   0.5   36.072978           82.59   
11        1      2                   0.9  224.473595           49.78   
12        1      3                   0.9  224.473595           4

### Summary of Performance for Athey's Synthetic Data

In [247]:
# SUMMARY: KALLUS on Athey
summary = {'Method': ['Dataset 1', 'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5', 'Dataset 1', 
                      'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5'],
           'Depth': [2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3],
           'P(Correct Treatment)': ['0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.9', '0.9',
                                   '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9'],
          'Error': [74.67, 74.67, 46.40, 79.86, 145, 109.21, 18.15, 18.15, 152.55, 90.47, 199.70, 238.67, 219.02, 
                    239.10, 201.21, 201.21, 99.23, 88.12, 196.07, 156.84],
          '% Classified': [71.97, 71.97, 78.49, 71.11, 61.22, 68.03, 88.33, 88.33, 60.41, 70.03, 53.35,
                          47.49, 49.91, 47.49, 53.42, 53.42, 71.19, 74.11, 54.12, 58.02]}
summary = pd.DataFrame(summary)
summary.to_csv('kallus_tree_athey.csv', index=False)

In [248]:
# SUMMARY: Bertsimas on Athey
summary = {'Method': ['Dataset 1', 'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5', 'Dataset 1', 
                      'Dataset 1', 'Dataset 2', 'Dataset 2', 'Dataset 3',
                     'Dataset 3', 'Dataset 4', 'Dataset 4', 'Dataset 5', 'Dataset 5'],
           'Depth': [2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3],
           'P(Correct Treatment)': ['0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.5', '0.9', '0.9',
                                   '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9', '0.9'],
          'Error': [74.67, 74.67, 46.40, 55.17, 145, 109.21, 18.15, 18.15, 152.55, 90.47, 199.70, 154.07, 219.02, 
                    180.83, 201.21, 116.75, 99.23, 88.12, 196.07, 156.84],
          '% Classified': [71.97, 71.97, 78.49, 76.49, 61.22, 68.03, 88.33, 88.33, 60.41, 70.03, 53.35,
                          62.17, 49.91, 53.91, 53.42, 71.59, 71.19, 74.11, 54.12, 58.02]}
summary = pd.DataFrame(summary)
summary.to_csv('bertsimas_tree_athey.csv', index=False)


In [37]:
def datapoint_tree_avg(node, i):
    if node in L: #if datapoint has reached leaf node, calculate error
        return node
    if train_X.iloc[i, branching[node]] <= 0: # go left (node 2)
        return datapoint_tree_avg(tree.get_left_children(node), i)
    else:
        return datapoint_tree_avg(tree.get_right_children(node), i)

In [ ]:
summation = {20: 0, 21: 0, 30: 0, 31: 0}
count = {20: 0, 21: 0, 30: 0, 31: 0}

for i in range(n):
    leaf_node = datapoint_tree_avg(1, i)
    index = str(leaf_node) + str(train_t[i])
    summation[int(index)] += train_y[i]
    count[int(index)] += 1
    
avg = {}
for i in summation:
    avg[i] = float(summation[i]) / count[i]

print(avg)

In [49]:
summation = {2: 0, 3: 0}
count = {2: 0, 3: 0}

for i in range(n):
    leaf_node = datapoint_tree_avg(1, i)
    if train_t[i] == treatments[leaf_node]:
        summation[leaf_node] += train_y[i]
        count[leaf_node] += 1
    
avg = {}
for i in summation:
    avg[i] = float(summation[i]) / count[i]

print(avg)

{2: 0.39851651102745433, 3: 0.34277517675515534}


In [59]:
N = L.union(L_c)
print(N)

model = gp.Model("Nathan")

# -- VARIABLE DECLARATION --

# -- Variables to determine: gamma and lambda --
# 1. gamma_p = choice of cut at node p ([0, 1]^C_p) (only applies to non-leaf node)
#       - represent with a matrix gamma (|L_c| x |C_p|)

gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 
# This assumes gamma is binary
#gamma = model.addVars(L_c, len(C), vtype=GRB.BINARY, name='gamma') 

# 2. lambda_pt = choice of treatment t at node p (only applies to leaf nodes L)
#       - represent with a matrix lamb (|L| x m)
lamb = model.addVars(L, m, vtype=GRB.BINARY, name='lamb')


# -- Other Variables in Formulation --
# 1. w_ip = membership of datapoint i in node p (only applies to leaf nodes L)
#       - represent with a matrix w (n x |L|)

c = model.addVars(n, N, vtype=GRB.BINARY, name='c')
# This assumes w is binary, when in reality it is continuous from 0-1
#w = model.addVars(n, L, vtype=GRB.BINARY, name='w') # Original paper has this be a continuous variable

# 2. mu_p = mean outcome of prescribed treatment in node p
#       - represent with a matrix mu (|L|)
v = model.addVars(n, L_c, vtype=GRB.BINARY, name='v') # define in constraint

# 3. nu_ip = "effect" of treatment in node p by multiplying mu and w
#       - represent with a matrix nu (n x |L|)
w = model.addVars(n, vtype=GRB.BINARY, name='w')

{1, 2, 3}


In [ ]:


"""if different_Cp:
# ---- K and Z if we followed the definition of C_p from the paper -----
    for p in L_c:
        k[p] = math.ceil(math.log2(len(C[p])))

    z = {}
    for p in L_c:
        matrix = np.zeros((k[p], len(C[p])))
        print(matrix.shape)
        for i in range(1, k[p]+1):
            for j in range(1, len(C[p])+1):
                if math.floor(j/(2**i)) % 2 == 1: # odd number
                    z[i-1, j-1] = 1
                else:
                    z[i-1, j-1] = 0
        z[p] = matrix

else:
    # ---- K and Z if we had constant C for all nodes ----
    k = math.ceil(math.log2(len(C)))
    z = np.zeros((k, len(C)))
    for i in range(1, k+1):
        for j in range(1, len(C)+1):
            if math.floor(j/(2**i)) % 2 == 1: # odd number
                z[i-1, j-1] = 1
            else:
                z[i-1, j-1] = 0
print(z)"""